In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

from matplotlib import pyplot as plt
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

In [2]:
ins_data = pd.read_pickle('../../data/feature engineered/insurance.pkl')
ins_data

,age,sex,bmi,children,smoker,region,charges
0,-0.854167,-1.0,-0.304319,-0.5,-0.590433,0.000000,0.475094
1,-1.020833,0.0,0.389592,0.0,0.000000,1.000000,-1.302772
2,-0.447917,0.0,0.302496,1.0,0.000000,1.000000,-0.585858
3,-0.239583,0.0,-0.987437,-0.5,0.000000,0.000000,0.692467
4,-0.281250,0.0,-0.183343,-0.5,0.000000,0.000000,-0.693955
...,...,...,...,...,...,...,...
1333,0.468750,0.0,0.067477,1.0,0.000000,0.000000,0.097725
1334,-1.020833,-1.0,0.178465,-0.5,0.000000,-0.025641,-1.119708
1335,-1.020833,-1.0,0.727647,-0.5,0.000000,1.000000,-1.345025
1336,-0.739583,-1.0,-0.571484,-0.5,0.000000,0.000000,-1.190043


In [3]:
Y = ins_data.loc[:, 'charges']
X = ins_data.drop('charges', axis=1)

In [4]:
X.shape, Y.shape

((1338, 6), (1338,))

In [5]:
freg_selection = SelectKBest(score_func=f_regression, k=4)
freg_selection.fit(X, Y)
df = freg_selection.transform(X)
df = pd.DataFrame(df, columns=['age', 'bmi', 'children', 'smoker'])
df

,age,bmi,children,smoker
0,-0.854167,-0.304319,-0.5,-0.590433
1,-1.020833,0.389592,0.0,0.000000
2,-0.447917,0.302496,1.0,0.000000
3,-0.239583,-0.987437,-0.5,0.000000
4,-0.281250,-0.183343,-0.5,0.000000
...,...,...,...,...
1333,0.468750,0.067477,1.0,0.000000
1334,-1.020833,0.178465,-0.5,0.000000
1335,-1.020833,0.727647,-0.5,0.000000
1336,-0.739583,-0.571484,-0.5,0.000000


****Ensemble Models****

In [6]:
from sklearn.preprocessing import PolynomialFeatures

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import StackingRegressor

from sklearn.model_selection import RepeatedKFold, cross_val_score
from sklearn.model_selection import GridSearchCV

In [7]:
cv = RepeatedKFold(n_splits=3, n_repeats=6, random_state=1)

***Bagging***

Bagging essentially helps reduce the variance of the entire ensemble by introducing
randomization into its construction procedure and is usually used with a base predictor
that has a tendency to overfit the training data. The primary point of consideration
here would be the stability (or lack thereof) of the training dataset: Bagging can
improve accuracy in cases where a slight perturbation in the data could result in a
significant change in the trained model.

In [8]:
bagging_grid = dict()
bagging_grid['base_estimator'] = [LinearRegression(),
                                  Ridge(alpha=0.1),
                                  KNeighborsRegressor(algorithm='kd_tree', n_neighbors=9),
                                  DecisionTreeRegressor(max_depth=5, min_samples_split=30)]
bagging_grid['max_samples'] = np.arange(0.1, 1.1, 0.1)
bagging_grid['max_features'] = np.arange(0.1, 1.1, 0.1)

In [9]:
bagging_model = BaggingRegressor()

bagging_grid_search = GridSearchCV(bagging_model, bagging_grid, cv=cv, scoring='r2')
bagging_scores = bagging_grid_search.fit(df, Y)

In [10]:
bagging_scores.best_score_, bagging_scores.best_estimator_

(0.8261960769033668,
 BaggingRegressor(base_estimator=DecisionTreeRegressor(max_depth=5,
                                                       min_samples_split=30),
                  max_samples=0.6))

***Random Forest***

In [11]:
rf_grid = dict()
rf_grid['criterion'] = ['mse', 'mae']
rf_grid['max_depth'] = list(range(1, 13))
rf_grid['min_samples_split'] = [25, 30, 35, 40, 50]

In [12]:
rf_model = RandomForestRegressor()

rf_grid_search = GridSearchCV(rf_model, rf_grid, cv=cv, scoring='r2')
rf_scores = rf_grid_search.fit(df, Y)

In [13]:
rf_scores.best_score_, rf_scores.best_estimator_

(0.8278797499155672, RandomForestRegressor(max_depth=4, min_samples_split=25))

***adaptive boosting***

In [28]:
adaboost_grid = dict()
adaboost_grid['base_estimator'] = [LinearRegression(),
                                   Ridge(alpha=0.1),
                                   KNeighborsRegressor(algorithm='kd_tree', n_neighbors=9),
                                   DecisionTreeRegressor(max_depth=5, min_samples_split=30)]
adaboost_grid['learning_rate'] = np.log(list(range(2, 101, 8)))
adaboost_grid['loss'] = ['linear', 'square', 'exponential']

In [29]:
adaboost_model = AdaBoostRegressor()

adaboost_grid_search = GridSearchCV(adaboost_model, adaboost_grid, cv=cv, scoring='r2')
adaboost_scores = adaboost_grid_search.fit(df, Y)

In [30]:
adaboost_scores.best_score_, adaboost_scores.best_estimator_

(0.7639922591311055,
 AdaBoostRegressor(base_estimator=Ridge(alpha=0.1),
                   learning_rate=2.8903717578961645))